# NIH Similarity Analysis
## $\color{grey}{\text{Author: Gaurav Anand}}$

## Import Libraries

In [84]:
import pandas as pd
import json
import sys
sys.path.append("/home/ga7er/.local/bin")

## Importing Data

In [128]:
# Survey Data
survey = pd.read_csv("../data/survey.csv", converters={'answers': eval})
# Question Bank
with open("../data/ques_ans_clean.json","r") as f:
    q_data = pd.DataFrame(json.load(f)) # all questions given

## Association Rules

In [129]:
import numpy as np
survey = survey.astype(str)
survey = survey.apply(lambda x: f"{x.name}_"+x,axis = 0)
# for column in survey.columns:
#    survey[column] = survey[column].apply(lambda x: np.nan if "nan" in x else x)
survey.head()

,user_id,Gender,Ethnicity,State,Age,Employment,0,1,2,3,...,169,170,171,172,173,174,175,176,177,178
0,user_id_0,Gender_Male,Ethnicity_White,State_Virginia,Age_Under 12 years old.,Employment_Military,0_nan,1_Every day,2_nan,3_nan,...,169_Almost everyday,170_At least once a week,171_Almost everyday,172_nan,173_nan,174_Organization again myself want month. Even...,175_Never,176_Yes,"177_['Friend' 'Spouse, partner, boyfriend, gir...",178_nan
1,user_id_1,Gender_Male,Ethnicity_Two or more races.,State_North Carolina,Age_45-54 years old.,Employment_A homemaker,0_nan,1_A few days (1-2 days),2_Every day,3_Most days (3-4 days),...,169_Almost everyday,170_A few times a month,171_Never,172_At least once a week,173_['Some other aspect of your physical appea...,174_Newspaper share fish party take particular...,175_2 - 5 times,176_Not Sure,"177_['Co-worker' 'Parent' 'Spouse, partner, bo...",178_Dinner energy spring western others analys...
2,user_id_2,Gender_Male,Ethnicity_Asian,State_Georgia,Age_12-17 years old.,Employment_Employed for wages,0_nan,1_None of the days (0 days),2_Most days (3-4 days),3_A few days (1-2 days),...,169_At least once a week,170_A few times a month,171_At least once a week,172_At least once a week,173_nan,174_Expect oil response piece although try. Or...,175_Once,176_nan,177_['Sibling (brother or sister)' 'Grandparen...,178_Kid inside box dream ready radio. Real com...
3,user_id_3,Gender_Male,Ethnicity_White,State_Colorado,Age_35-44 years old.,Employment_A homemaker,0_nan,1_nan,2_None of the days (0 days),3_None of the days (0 days),...,169_A few times a month,170_nan,171_nan,172_A few times a month,173_['Your religion' 'Your ancestry or nationa...,174_Open water even however once. Various upon...,175_Never,176_Not Sure,177_['Co-worker'],178_Girl sense learn page expert fine single b...
4,user_id_4,Gender_Female,Ethnicity_Black or African American,State_Maryland,Age_25-34 years old.,Employment_Self-employed,0_nan,1_Most days (3-4 days),2_None of the days (0 days),3_Most days (3-4 days),...,169_Almost everyday,170_Almost everyday,171_Almost everyday,172_A few times a month,173_['Your ancestry or national origins' 'Your...,174_Support certainly order great paper. Easy ...,175_2 - 5 times,176_Not Sure,177_nan,178_Now usually article important join task wh...


In [122]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
# Data preprocessing
q_data["Answer Type"] = q_data["Answer Type"].str.lower()
questions = list(q_data.query("`Answer Type` == 'single-select'").qid.astype(str))
questions.extend(["user_id","Gender","Ethnicity","State","Age","Employment"])
survey_treated = survey[questions]
survey_treated = survey_treated.astype(str)
survey_treated = survey_treated.drop(["user_id","97","109","115","0"],axis=1).to_numpy().tolist()

te = TransactionEncoder()
te_ary = te.fit(survey_treated).transform(survey_treated)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = fpgrowth(df,min_support=0.2,use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.51,(45_Yes)
1,0.46,(139_Yes)
2,0.45,(154_No)
3,0.45,(136_Yes)
4,0.43,(Gender_Male)
...,...,...
402,0.21,"(89_Yes, 11_No)"
403,0.20,"(89_Yes, 44_No)"
404,0.21,"(89_Yes, 154_No)"
405,0.21,"(93_No, 80_Fairly Often)"


In [133]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets,metric='lift',support_only=False,min_threshold=1.2,)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Gender_Male),(44_No),0.43,0.43,0.24,0.558140,1.297999,0.0551,1.290000
1,(44_No),(Gender_Male),0.43,0.43,0.24,0.558140,1.297999,0.0551,1.290000
2,(130_Yes),(150_Yes),0.37,0.40,0.21,0.567568,1.418919,0.0620,1.387500
3,(150_Yes),(130_Yes),0.40,0.37,0.21,0.525000,1.418919,0.0620,1.326316
4,(159_The same as usual),(11_No),0.33,0.49,0.21,0.636364,1.298701,0.0483,1.402500
5,(11_No),(159_The same as usual),0.49,0.33,0.21,0.428571,1.298701,0.0483,1.172500
6,(93_No),(154_Yes),0.49,0.39,0.23,0.469388,1.203558,0.0389,1.149615
7,(154_Yes),(93_No),0.39,0.49,0.23,0.589744,1.203558,0.0389,1.243125
8,(85_Yes),(93_No),0.36,0.49,0.22,0.611111,1.247166,0.0436,1.311429
9,(93_No),(85_Yes),0.49,0.36,0.22,0.448980,1.247166,0.0436,1.161481


From the association rules seen above, we see that there is a positive correlation between 
* A male respondent answering no to being on chemotherapy or immunosuppresants
* A respondent smoking cocaine and also smoking heroin
* and many more correlations ...

This rule mining is an example of how we can use association rules to generate insights between respondents. 